# Gröbner Basis Complexity Estimate For Arion & ArionHash

In this SageMath notebook we estimate the complexity of Gröbner basis attacks on Arion & ArionHash.

In [1]:
def log2(x):
    return log(x) / log(2)

def stirling_approximation(n, k):
    """
    Computes the logarithm in base 2 of Stirling's binomial coefficient approximation.
    """
    approx_1 = n - k
    approx_1 *= log2(n / (n - k))
    approx_1 += k * log(n / k)
    approx_2 = log2(n)
    approx_2 -= log2(pi)
    approx_2 -= log2(k)
    approx_2 -= log2(n - k)
    approx_2 /= 2
    return approx_1 + approx_2

In [2]:
def complexity_deterministic_solving(n, d, q):
    compl = sqrt(n) * d**(2 + (n - 1) / n)
    compl += d * log(q) * log(d) * log(log(d))
    compl += d * log(d)**2 * log(log(d))
    compl = log2(compl)
    return float(compl)

def complexity_probabilistic_solving(n, d, q, omega=2):
    compl = n * d**omega
    if d < q:
        compl += d * log(q) * log(d) * log(log(d))
        compl += d * log(d)**2 * log(log(d))
    else:
        compl += q * log(d) * log(q) * log(log(q))
        compl += q * log(q)**2 * log(log(q))
    compl = log2(compl)
    return float(compl)

In [3]:
def complexity_gb_computation(n, d, omega=2, stirling=False):
    if stirling:
        return float(omega * stirling_approximation(n + d, d))
    else:
        return float(omega * log2(binomial(n + d, d)))

In [4]:
def arion_macaulay_bound(n, r, d_1, d_2):
    out = d_2 + 1
    for i in range(1, n):
        out += 2**(n - i) * (d_1 + 1) - d_1
    out *= r
    out += 1
    out -= r * (n + 1)
    return out

In [5]:
def arion_GTDS_maximal_degree(n, d_1, d_2):
    return 2**(n - 1) * (d_1 + d_2) - d_1

### Table Legend

 - GB_MB ... complexity of Gröbner basis computation if Macaulay bound is used as solving degree
 - GB_min ... complexity of Gröbner basis computation if highest degree in polynomial system is used as solving degree
 - Solving_det ... complexity of polynomial system solving with deterministic algorithm
 - Solving_prob ... complexity of polynomial system solving with probabilitstic algorithm

All complexities are given in bits.

## Arion

We hypothesize that the quotient space dimension of Arion is

$$ \dim_{\mathbb{F}_p} \left( \mathcal{F}_\textsf{Arion} \right) (n, r, d_1, d_2) = \left( d_2 \cdot \left( d_1 + 2 \right)^{n - 1} \right)^r, \qquad n \geq 1. $$

In [6]:
def arion_quotient_space_dimension(n, r, d_1, d_2):
    return (d_2 * (d_1 + 2)**(n - 1))**r

In [7]:
rounds = 12
branches = 8
sizes = [60, 120, 250] # 2^N

omega = 2
security_bound = 128

print("Arion", "\n")
print("Minimum number of rounds for", security_bound, "bit security for system solving.", "\n")
print("omega:", omega, "\n")

print("N", "\t",
      "n", "\t", 
      "r", "\t",
      "d_1", "\t", 
      "d_2", "\t",  
      "GB_min", "\t", 
      "GB_MB", "\t",
      "Solving_det", "\t", 
      "Solving_prob")
for N in sizes:
    q = 2**N
    for n in range(3, branches + 1):
        d_1 = 3
        d_2 = 121
        r = 1
        n_vars = (n + 1) * r
        sd_mb = arion_macaulay_bound(n, r, d_1, d_2)              # solvind degree Macaulay bound
        sd_min = max([d_2, arion_GTDS_maximal_degree(n, d_1, 1)]) # minimal possible solving degree
        d = arion_quotient_space_dimension(n, r, d_1, d_2)
        compl_solv_det = floor(complexity_deterministic_solving(n_vars, d, q))
        compl_solv_prob = floor(complexity_probabilistic_solving(n_vars, d, q, omega))
        if compl_solv_det >= security_bound or compl_solv_prob >= security_bound:
            compl_gb_mb = floor(complexity_gb_computation(n_vars, sd_mb, omega, True))
            compl_gb_min = floor(complexity_gb_computation(n_vars, sd_min, omega, True))
            print(N, "\t",
                  n, "\t", 
                  r, "\t", 
                  d_1, "\t", 
                  d_2, "\t", 
                  compl_gb_min, "\t\t",
                  compl_gb_mb, "\t",
                  compl_solv_det, "\t\t",
                  compl_solv_prob)
        while compl_solv_det < security_bound or compl_solv_prob < security_bound:
            r += 1
            n_vars = (n + 1) * r
            d = arion_quotient_space_dimension(n, r, d_1, d_2)
            compl_solv_det = floor(complexity_deterministic_solving(n_vars, d, q))
            compl_solv_prob = floor(complexity_probabilistic_solving(n_vars, d, q, omega))
            if compl_solv_det >= security_bound or compl_solv_prob >= security_bound:
                compl_gb_mb = floor(complexity_gb_computation(n_vars, sd_mb, omega, True))
                compl_gb_min = floor(complexity_gb_computation(n_vars, sd_min, omega, True))
                print(N, "\t",
                      n, "\t", 
                      r, "\t", 
                      d_1, "\t", 
                      d_2, "\t", 
                      compl_gb_min, "\t\t",
                      compl_gb_mb, "\t",
                      compl_solv_det, "\t\t",
                      compl_solv_prob)
                
        d_1 = 5
        d_2 = 121
        r = 1
        n_vars = (n + 1) * r
        sd_mb = arion_macaulay_bound(n, r, d_1, d_2)              # solvind degree Macaulay bound
        sd_min = max([d_2, arion_GTDS_maximal_degree(n, d_1, 1)]) # minimal possible solving degree
        d = arion_quotient_space_dimension(n, r, d_1, d_2)
        compl_solv_det = floor(complexity_deterministic_solving(n_vars, d, q))
        compl_solv_prob = floor(complexity_probabilistic_solving(n_vars, d, q, omega))
        if compl_solv_det >= security_bound or compl_solv_prob >= security_bound:
            compl_gb_mb = floor(complexity_gb_computation(n_vars, sd_mb), omega, True)
            compl_gb_min = floor(complexity_gb_computation(n_vars, sd_min), omega, True)
            print(N, "\t",
                  n, "\t", 
                  r, "\t", 
                  d_1, "\t", 
                  d_2, "\t", 
                  compl_gb_min, "\t\t",
                  compl_gb_mb, "\t",
                  compl_solv_det, "\t\t",
                  compl_solv_prob)
        while compl_solv_det < security_bound or compl_solv_prob < security_bound:
            r += 1
            n_vars = (n + 1) * r
            d = arion_quotient_space_dimension(n, r, d_1, d_2)
            compl_solv_det = floor(complexity_deterministic_solving(n_vars, d, q))
            compl_solv_prob = floor(complexity_probabilistic_solving(n_vars, d, q, omega))
            if compl_solv_det >= security_bound or compl_solv_prob >= security_bound:
                compl_gb_mb = floor(complexity_gb_computation(n_vars, sd_mb, omega, True))
                compl_gb_min = floor(complexity_gb_computation(n_vars, sd_min, omega, True))
                print(N, "\t",
                      n, "\t", 
                      r, "\t", 
                      d_1, "\t", 
                      d_2, "\t", 
                      compl_gb_min, "\t\t",
                      compl_gb_mb, "\t",
                      compl_solv_det, "\t\t",
                      compl_solv_prob)
        print("")

Arion 

Minimum number of rounds for 128 bit security for system solving. 

omega: 2 

N 	 n 	 r 	 d_1 	 d_2 	 GB_min 	 GB_MB 	 Solving_det 	 Solving_prob
60 	 3 	 4 	 3 	 121 	 123 		 129 	 137 		 96
60 	 3 	 5 	 3 	 121 	 143 		 150 	 172 		 119
60 	 3 	 6 	 3 	 121 	 162 		 170 	 207 		 143
60 	 3 	 4 	 5 	 121 	 123 		 131 	 149 		 104
60 	 3 	 5 	 5 	 121 	 143 		 153 	 187 		 129

60 	 4 	 4 	 3 	 121 	 143 		 160 	 165 		 115
60 	 4 	 5 	 3 	 121 	 166 		 186 	 207 		 143
60 	 4 	 3 	 5 	 121 	 118 		 135 	 136 		 95
60 	 4 	 4 	 5 	 121 	 143 		 167 	 183 		 127
60 	 4 	 5 	 5 	 121 	 166 		 195 	 229 		 158

60 	 5 	 3 	 3 	 121 	 134 		 164 	 145 		 101
60 	 5 	 4 	 3 	 121 	 162 		 201 	 194 		 134
60 	 5 	 3 	 5 	 121 	 134 		 174 	 162 		 113
60 	 5 	 4 	 5 	 121 	 162 		 215 	 217 		 149

60 	 6 	 3 	 3 	 121 	 150 		 208 	 166 		 115
60 	 6 	 4 	 3 	 121 	 181 		 257 	 222 		 153
60 	 6 	 3 	 5 	 121 	 172 		 225 	 187 		 130

60 	 7 	 3 	 3 	 121 	 209 		 266 	 187 		 1

## ArionHash

We hypothesize that the quotient space dimension of ArionHash is


$$ \dim_{\mathbb{F}_p} \left( \mathcal{F}_\textsf{ArionHash} \right) (n, r, d_1, d_2) = \Big( 2^{n - 1} \cdot d_2 \cdot \left( d_1 + 1 \right) - d_1 \cdot d_2 \Big)^r, \qquad n \geq 1. $$

In [8]:
def arion_hash_quotient_space_dimension(n, r, d_1, d_2):
    return (2**(n - 1) * d_2 * (d_1 + 1) - d_1 * d_2)**r

In [9]:
rounds = 12
branches = 8
sizes = [60, 120, 250] # 2^N

omega = 2
security_bound = 128

print("ArionHash", "\n")
print("Minimum number of rounds for", security_bound, "bit security for system solving.", "\n")
print("omega:", omega, "\n")

print("N", "\t",
      "n", "\t", 
      "r", "\t",
      "d_1", "\t", 
      "d_2", "\t",  
      "GB_min", "\t",
      "GB_MB", "\t", 
      "Solving_det", "\t", 
      "Solving_prob")
for N in sizes:
    q = 2**N
    for n in range(3, branches + 1):
        d_1 = 3
        d_2 = 121
        r = 1
        n_vars = (n + 1) * r
        sd_mb = arion_macaulay_bound(n, r, d_1, d_2)              # solvind degree Macaulay bound
        sd_min = max([d_2, arion_GTDS_maximal_degree(n, d_1, 1)]) # minimal possible solving degree
        d = arion_hash_quotient_space_dimension(n, r, d_1, d_2)
        compl_solv_det = floor(complexity_deterministic_solving(n_vars, d, q))
        compl_solv_prob = floor(complexity_probabilistic_solving(n_vars, d, q, omega))
        if compl_solv_det >= security_bound or compl_solv_prob >= security_bound:
            compl_gb_mb = floor(complexity_gb_computation(n_vars, sd_mb, omega, True))
            compl_gb_min = floor(complexity_gb_computation(n_vars, sd_min, omega, True))
            print(N, "\t",
                  n, "\t", 
                  r, "\t", 
                  d_1, "\t", 
                  d_2, "\t", 
                  compl_gb_min, "\t\t",
                  compl_gb_mb, "\t",
                  compl_solv_det, "\t\t",
                  compl_solv_prob)
        while compl_solv_det < security_bound or compl_solv_prob < security_bound:
            r += 1
            n_vars = (n + 1) * r
            d = arion_hash_quotient_space_dimension(n, r, d_1, d_2)
            compl_solv_det = floor(complexity_deterministic_solving(n_vars, d, q))
            compl_solv_prob = floor(complexity_probabilistic_solving(n_vars, d, q, omega))
            if compl_solv_det >= security_bound or compl_solv_prob >= security_bound:
                compl_gb_mb = floor(complexity_gb_computation((n + 1) * r, sd_mb, omega, True))
                compl_gb_min = floor(complexity_gb_computation((n + 1) * r, sd_min, omega, True))
                print(N, "\t",
                      n, "\t", 
                      r, "\t", 
                      d_1, "\t", 
                      d_2, "\t", 
                      compl_gb_min, "\t\t",
                      compl_gb_mb, "\t",
                      compl_solv_det, "\t\t",
                      compl_solv_prob)
                
        d_1 = 5
        d_2 = 121
        r = 1
        n_vars = (n + 1) * r
        sd_mb = arion_macaulay_bound(n, r, d_1, d_2)              # solvind degree Macaulay bound
        sd_min = max([d_2, arion_GTDS_maximal_degree(n, d_1, 1)]) # minimal possible solving degree
        d = arion_hash_quotient_space_dimension(n, r, d_1, d_2)
        compl_solv_det = floor(complexity_deterministic_solving(n_vars, d, q))
        compl_solv_prob = floor(complexity_probabilistic_solving(n_vars, d, q, omega))
        if compl_solv_det >= security_bound or compl_solv_prob >= security_bound:
            compl_gb_mb = floor(complexity_gb_computation(n_vars, sd_mb), omega, True)
            compl_gb_min = floor(complexity_gb_computation(n_vars, sd_min), omega, True)
            print(N, "\t",
                  n, "\t", 
                  r, "\t", 
                  d_1, "\t", 
                  d_2, "\t", 
                  compl_gb_min, "\t\t",
                  compl_gb_mb, "\t",
                  compl_solv_det, "\t\t",
                  compl_solv_prob)
        while compl_solv_det < security_bound or compl_solv_prob < security_bound:
            r += 1
            n_vars = (n + 1) * r
            d = arion_hash_quotient_space_dimension(n, r, d_1, d_2)
            compl_solv_det = floor(complexity_deterministic_solving(n_vars, d, q))
            compl_solv_prob = floor(complexity_probabilistic_solving(n_vars, d, q, omega))
            if compl_solv_det >= security_bound or compl_solv_prob >= security_bound:
                compl_gb_mb = floor(complexity_gb_computation(n_vars, sd_mb, omega, True))
                compl_gb_min = floor(complexity_gb_computation(n_vars, sd_min, omega, True))
                print(N, "\t",
                      n, "\t", 
                      r, "\t", 
                      d_1, "\t", 
                      d_2, "\t", 
                      compl_gb_min, "\t\t",
                      compl_gb_mb, "\t",
                      compl_solv_det, "\t\t",
                      compl_solv_prob)
        print("")

ArionHash 

Minimum number of rounds for 128 bit security for system solving. 

omega: 2 

N 	 n 	 r 	 d_1 	 d_2 	 GB_min 	 GB_MB 	 Solving_det 	 Solving_prob
60 	 3 	 5 	 3 	 121 	 143 		 150 	 158 		 110
60 	 3 	 6 	 3 	 121 	 162 		 170 	 190 		 132
60 	 3 	 4 	 5 	 121 	 123 		 131 	 133 		 93
60 	 3 	 5 	 5 	 121 	 143 		 153 	 166 		 115
60 	 3 	 6 	 5 	 121 	 162 		 173 	 200 		 138

60 	 4 	 4 	 3 	 121 	 143 		 160 	 141 		 98
60 	 4 	 5 	 3 	 121 	 166 		 186 	 176 		 122
60 	 4 	 6 	 3 	 121 	 187 		 210 	 212 		 146
60 	 4 	 4 	 5 	 121 	 143 		 167 	 147 		 103
60 	 4 	 5 	 5 	 121 	 166 		 195 	 185 		 128

60 	 5 	 4 	 3 	 121 	 162 		 201 	 154 		 107
60 	 5 	 5 	 3 	 121 	 187 		 235 	 193 		 133
60 	 5 	 4 	 5 	 121 	 162 		 215 	 161 		 111
60 	 5 	 5 	 5 	 121 	 187 		 251 	 201 		 139

60 	 6 	 4 	 3 	 121 	 181 		 257 	 167 		 115
60 	 6 	 5 	 3 	 121 	 208 		 301 	 208 		 143
60 	 6 	 3 	 5 	 121 	 172 		 225 	 130 		 91
60 	 6 	 4 	 5 	 121 	 210 		 279 	 173 		